### Model Predictive Controller (MPC) Design



In [31]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import fractional_matrix_power

A1 = np.matrix([[0.9347, 0.5194],[0.3835, 0.8310]])
A2 = np.matrix([[0.0591, 0.2641],[1.7971, 0.8717]])

B = np.matrix([[-1.4462],[-0.7012]])

n=A1.shape[0]
b=B.shape[1]

P = cp.Variable((n,n), PSD=True)
Y = cp.Variable((b,n))
gamma = cp.Variable((1,1))
I = np.identity(n)
Ib = np.identity(b)
Zn = np.zeros((n,n))
Znb = np.zeros((n,b))

Q=I;
R=np.identity(b)
Qf = fractional_matrix_power(Q, 0.5)
Rf = fractional_matrix_power(R, 0.5)

LMI1 = [[P, P*A1.T+Y.T*B.T, P*Qf, Y.T*Rf],
        [(P*A1.T+Y.T*B.T).T, P, Zn, Znb],
        [(P*Qf).T, Zn, gamma*I, Znb],
        [(Y.T*Rf).T, Znb.T, Znb.T, gamma*Ib]]


